In [ ]:
!pip install -q --upgrade pip setuptools
!pip install -q "numpy~=1.26.4"
!pip install -q ultralytics openvino-dev tqdm requests PyYAML

import torch
from ultralytics import YOLO
from pathlib import Path
import os
import shutil
import json
from tqdm import tqdm
import time
import numpy as np

print(f"Torch version: {torch.__version__}")
print(f"NumPy version: {np.__version__}")
try:
    from openvino.runtime import Core
    print(f"OpenVINO version: {Core().get_versions('CPU')['CPU']}")
    import networkx as nx
    print(f"NetworkX version: {nx.__version__}")
except ImportError as e:
    print(f"One of the core libraries (OpenVINO, NetworkX) not fully imported, check installation: {e}")
except Exception as e:
    print(f"An error occurred during import or version check: {e}")

In [ ]:
!mkdir -p /content/my_coco_data/images

In [ ]:
# ielādē datu kopas vispirms
!unzip /content/val2017.zip -d /content/my_coco_data/images/
!unzip /content/annotations_trainval2017.zip -d /content/my_coco_data

In [ ]:
DATASET_ROOT_DIR = "/content/my_coco_data"

MODEL_PT = 'yolo11x.pt'
EXPORT_DIR_BASE = 'exported_models_coco_val2017'

IMAGES_SUBDIR_RELATIVE = "images/val2017"
JSON_ANNOTATION_SUBDIR_RELATIVE = "annotations/instances_val2017.json"
LABELS_SUBDIR_RELATIVE = "labels/val2017"

ABS_IMAGES_DIR = Path(DATASET_ROOT_DIR) / IMAGES_SUBDIR_RELATIVE
ABS_JSON_ANNOTATION_FILE = Path(DATASET_ROOT_DIR) / JSON_ANNOTATION_SUBDIR_RELATIVE
ABS_LABELS_DIR = Path(DATASET_ROOT_DIR) / LABELS_SUBDIR_RELATIVE

DATASET_YAML_NAME = "coco_val2017_for_yolo.yaml"
ABS_DATASET_YAML_PATH = f"/content/{DATASET_YAML_NAME}"

COCO_CLASSES = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
    'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
    'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
    'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
    'hair drier', 'toothbrush'
]

In [ ]:
def convert_coco_json_to_yolo_txt(json_annotation_path_abs, image_dir_path_abs, output_label_dir_path_abs, class_names_list):
    if not json_annotation_path_abs.is_file():
        print(f"ERROR: JSON Annotation file not found at {json_annotation_path_abs}.")
        return False
    os.makedirs(output_label_dir_path_abs, exist_ok=True)
    try:
        with open(json_annotation_path_abs, 'r') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error loading JSON file {json_annotation_path_abs}: {e}")
        return False
    if 'categories' not in data or 'images' not in data or 'annotations' not in data:
        print(f"ERROR: JSON file {json_annotation_path_abs} is missing 'categories', 'images', or 'annotations' key.")
        return False

    coco_id_to_name_from_json = {cat['id']: cat['name'] for cat in data['categories']}
    name_to_target_id = {name: i for i, name in enumerate(class_names_list)}
    coco_cat_id_to_target_id = {}
    for coco_id, coco_name in coco_id_to_name_from_json.items():
        if coco_name in name_to_target_id:
            coco_cat_id_to_target_id[coco_id] = name_to_target_id[coco_name]

    image_info_map = {img['id']: {'file_name': img['file_name'],
                                  'width': img['width'],
                                  'height': img['height']}
                      for img in data['images']}
    annotations_per_image = {}
    for ann in data['annotations']:
        image_id = ann['image_id']
        if image_id not in annotations_per_image:
            annotations_per_image[image_id] = []
        annotations_per_image[image_id].append(ann)
    files_created_count = 0
    for image_id, anns_list in tqdm(annotations_per_image.items(), desc=f"Converting {json_annotation_path_abs.name}", leave=False):
        if image_id not in image_info_map:
            continue
        img_data = image_info_map[image_id]
        img_filename_base = Path(img_data['file_name']).stem
        img_width = img_data['width']
        img_height = img_data['height']
        if img_width == 0 or img_height == 0:
            continue
        yolo_label_path = output_label_dir_path_abs / f"{img_filename_base}.txt"
        with open(yolo_label_path, 'w') as f_out:
            for ann_data in anns_list:
                coco_cat_id = ann_data['category_id']
                if coco_cat_id in coco_cat_id_to_target_id:
                    target_class_id = coco_cat_id_to_target_id[coco_cat_id]
                    bbox_coco = ann_data['bbox']
                    x_min, y_min, w, h = bbox_coco
                    x_center = x_min + w / 2
                    y_center = y_min + h / 2
                    x_center_norm = x_center / img_width
                    y_center_norm = y_center / img_height
                    width_norm = w / img_width
                    height_norm = h / img_height
                    f_out.write(f"{target_class_id} {x_center_norm:.6f} {y_center_norm:.6f} {width_norm:.6f} {height_norm:.6f}\n")
        files_created_count += 1
    if files_created_count == 0 and len(annotations_per_image) > 0 :
        print("WARNING: No label files were created. Check paths, image ID matching, and category mapping.")
    return True

In [ ]:
if os.path.exists(ABS_LABELS_DIR):
    shutil.rmtree(ABS_LABELS_DIR)
yolo_cache_file = ABS_LABELS_DIR.parent / (ABS_LABELS_DIR.name + ".cache")
if yolo_cache_file.exists():
    os.remove(yolo_cache_file)
yolo_cache_file_inside = ABS_LABELS_DIR / (ABS_LABELS_DIR.name + ".cache")
if yolo_cache_file_inside.exists():
     os.remove(yolo_cache_file_inside)

conversion_successful = convert_coco_json_to_yolo_txt(
    ABS_JSON_ANNOTATION_FILE,
    ABS_IMAGES_DIR,
    ABS_LABELS_DIR,
    COCO_CLASSES
)

if conversion_successful:
    yaml_content = f"""
path: {DATASET_ROOT_DIR}
train: {IMAGES_SUBDIR_RELATIVE}
val: {IMAGES_SUBDIR_RELATIVE}

names:
"""
    for i, name in enumerate(COCO_CLASSES):
        yaml_content += f"  {i}: {name}\n"
    try:
        with open(ABS_DATASET_YAML_PATH, 'w') as f:
            f.write(yaml_content)
    except Exception as e:
        print(f"ERROR: Failed to write YAML file: {e}")
        ABS_DATASET_YAML_PATH = None
else:
    print("ERROR: Label conversion failed. YAML file will not be created.")
    ABS_DATASET_YAML_PATH = None

In [ ]:
results_summary = {}
yolo_base_model = None

def validate_and_store_results(model_to_eval, model_id_str, data_yaml_path, device_str='cpu'):
    global results_summary
    metrics_data = {
        "mAP50-95": float('nan'), "mAP50": float('nan'), "mAP75": float('nan'),
        "Precision": float('nan'), "Recall": float('nan'), "F1": float('nan'),
        "time_ms": float('nan')
    }
    try:
        start_val_time = time.time()
        if not isinstance(model_to_eval, YOLO):
            model_to_eval = YOLO(str(model_to_eval))

        metrics_obj = model_to_eval.val(data=data_yaml_path, split='val', device=device_str, plots=False, batch=1)
        end_val_time = time.time()

        if metrics_obj and hasattr(metrics_obj, 'box') and hasattr(metrics_obj.box, 'map'):
            metrics_data["mAP50-95"] = metrics_obj.box.map
            metrics_data["mAP50"] = metrics_obj.box.map50
            metrics_data["mAP75"] = metrics_obj.box.map75
            if hasattr(metrics_obj.box, 'p') and isinstance(metrics_obj.box.p, np.ndarray) and metrics_obj.box.p.size > 0:
                metrics_data["Precision"] = np.mean(metrics_obj.box.p)
            elif hasattr(metrics_obj.box, 'p'):
                metrics_data["Precision"] = metrics_obj.box.p
            if hasattr(metrics_obj.box, 'r') and isinstance(metrics_obj.box.r, np.ndarray) and metrics_obj.box.r.size > 0:
                metrics_data["Recall"] = np.mean(metrics_obj.box.r)
            elif hasattr(metrics_obj.box, 'r'):
                metrics_data["Recall"] = metrics_obj.box.r
            if hasattr(metrics_obj.box, 'f1') and isinstance(metrics_obj.box.f1, np.ndarray) and metrics_obj.box.f1.size > 0:
                metrics_data["F1"] = np.mean(metrics_obj.box.f1)
            elif hasattr(metrics_obj.box, 'f1'):
                 metrics_data["F1"] = metrics_obj.box.f1
        if hasattr(metrics_obj, 'speed') and 'inference' in metrics_obj.speed:
            metrics_data["time_ms"] = metrics_obj.speed['preprocess'] + metrics_obj.speed['inference'] + metrics_obj.speed['postprocess']
        else:
             num_images = len(metrics_obj.dataset.im_files) if hasattr(metrics_obj, 'dataset') and hasattr(metrics_obj.dataset, 'im_files') else 0
             if num_images > 0:
                metrics_data["time_ms"] = ((end_val_time - start_val_time) / num_images) * 1000
    except Exception as e:
        print(f"ERROR during validation of {model_id_str}: {e}")
        import traceback
        traceback.print_exc()
    results_summary[model_id_str] = metrics_data

if ABS_DATASET_YAML_PATH and Path(ABS_DATASET_YAML_PATH).exists():
    yolo_base_model = YOLO(MODEL_PT)

    if yolo_base_model:
        try:
            fp32_export_path_obj_dir = yolo_base_model.export(
                format='openvino', half=False, project=EXPORT_DIR_BASE, name=f"{Path(MODEL_PT).stem}_ov_fp32"
            )
            fp32_ov_model_path = Path(fp32_export_path_obj_dir) / (Path(MODEL_PT).stem + ".xml")
            if not fp32_ov_model_path.is_file():
                found_xmls = list(Path(fp32_export_path_obj_dir).glob("*.xml"))
                if found_xmls: fp32_ov_model_path = found_xmls[0]
                else: raise FileNotFoundError(f"OpenVINO FP32 XML not found in {fp32_export_path_obj_dir}")

            validate_and_store_results(str(fp32_ov_model_path), "OpenVINO FP32", ABS_DATASET_YAML_PATH, device_str='cpu')
        except Exception as e:
            print(f"ERROR during OpenVINO FP32: {e}")
            results_summary["OpenVINO FP32"] = {"mAP50-95": float('nan'), "mAP50": float('nan'), "mAP75": float('nan'), "Precision": float('nan'), "Recall": float('nan'), "F1": float('nan'), "time_ms": float('nan')}

    if yolo_base_model:
        try:
            int8_export_path_obj_dir = yolo_base_model.export(
                format='openvino', int8=True, data=ABS_DATASET_YAML_PATH, project=EXPORT_DIR_BASE, name=f"{Path(MODEL_PT).stem}_ov_int8"
            )
            int8_ov_model_path = Path(int8_export_path_obj_dir) / (Path(MODEL_PT).stem + ".xml")
            if not int8_ov_model_path.is_file():
                found_xmls = list(Path(int8_export_path_obj_dir).glob("*.xml"))
                if found_xmls: int8_ov_model_path = found_xmls[0]
                else: raise FileNotFoundError(f"OpenVINO INT8 XML not found in {int8_export_path_obj_dir}")

            validate_and_store_results(str(int8_ov_model_path), "OpenVINO INT8", ABS_DATASET_YAML_PATH, device_str='cpu')
        except Exception as e:
            print(f"ERROR during OpenVINO INT8: {e}")
            results_summary["OpenVINO INT8"] = {"mAP50-95": float('nan'), "mAP50": float('nan'), "mAP75": float('nan'), "Precision": float('nan'), "Recall": float('nan'), "F1": float('nan'), "time_ms": float('nan')}
else:
    print("ERROR: Dataset YAML path is not valid. Cannot proceed with evaluations.")

In [ ]:
print("\n--- Overall Performance Summary (val2017 on CPU) ---")
header = (f"{'Model':<25} | {'mAP@.5':<10} | {'mAP@.75':<10} | {'mAP@.5-.95':<12} | "
          f"{'Precision':<10} | {'Recall':<10} | {'F1-score':<10} | "
          f"{'Time (ms/img)':<15} | {'Speed-up vs PT FP32':<20}")
separator = "-" * (len(header) + 5)
print(header)
print(separator)

baseline_time_ms = results_summary.get("PyTorch FP32", {}).get("time_ms", float('nan'))
if np.isnan(baseline_time_ms) and "PyTorch FP32" not in results_summary:
    print("NOTE: PyTorch FP32 baseline not run or metrics unavailable. Speed-up calculations will be N/A.")

model_order = ["PyTorch FP32", "OpenVINO FP32", "OpenVINO INT8"]
evaluated_models_in_order = [m for m in model_order if m in results_summary]
remaining_models = [m for m in results_summary if m not in evaluated_models_in_order]
final_print_order = evaluated_models_in_order + remaining_models

for model_name in final_print_order:
    metrics = results_summary.get(model_name, {})
    map_val = metrics.get("mAP50-95", float('nan'))
    map50_val = metrics.get("mAP50", float('nan'))
    map75_val = metrics.get("mAP75", float('nan'))
    precision_val = metrics.get("Precision", float('nan'))
    recall_val = metrics.get("Recall", float('nan'))
    f1_val = metrics.get("F1", float('nan'))
    time_val = metrics.get("time_ms", float('nan'))

    speed_up_str = "N/A"
    if model_name == "PyTorch FP32" and not np.isnan(time_val):
        speed_up_str = "Baseline"
    elif "PyTorch FP32" in results_summary and not np.isnan(baseline_time_ms) and baseline_time_ms > 0 and not np.isnan(time_val) and time_val > 0:
        speed_up = baseline_time_ms / time_val
        speed_up_str = f"{speed_up:.2f}x"

    print(f"{model_name:<25} | {map50_val:<10.4f} | {map75_val:<10.4f} | {map_val:<12.4f} | "
          f"{precision_val:<10.4f} | {recall_val:<10.4f} | {f1_val:<10.4f} | "
          f"{time_val:<15.2f} | {speed_up_str:<20}")

if not results_summary:
    print("No results found in results_summary to display.")